In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
import urllib

In [9]:
tf.reset_default_graph()

In [11]:
with urllib.request.urlopen("https://airhive.it/dataset") as url:
    df = pd.read_json(url)

ValueError: unknown url type: 'airhive.it/dataset'

In [12]:
n_steps = 30
n_inputs = 1
n_neurons = 100
n_outputs = 1

In [ ]:
resolution = 1
lun = len(data) - 100

def time_series(t):
    array = []
    for x in t:
        array.append(df.iloc[x].values)
    return np.array(array).copy()

def next_batch(batch_size, n_steps):
    t0 = np.random.randint(lun, size = (batch_size, 1))
    Ts = t0 + np.arange(0, n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, n_inputs), ys[:, 1:].reshape(-1, n_steps, n_inputs)

In [4]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [5]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [6]:
saver = tf.train.Saver()

In [7]:
n_iterations = 2000
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    saver.save(sess, "./modello_RNN")

NameError: name 'next_batch' is not defined